In [5]:
import math
import numpy as np
import cv2
import os
import winsound
duration = 1000  # milliseconds
freq = 440  # Hz
# import yolo_preprocessing
# from yolo_preprocessing import read_annotations

In [6]:
 def draw_on_image(imageLoc,objectLis):
    image_path = imageLoc
    image = cv2.imread(image_path)
    for shaft in objectLis:
        x0_floored = int(np.floor(shaft["x0"]))
        y0_floored = int(np.floor(shaft["y0"]))
        x1_floored = int(np.floor(shaft["x1"]))
        y1_floored = int(np.floor(shaft["y1"]))
        cv2.circle(image, (x0_floored, y0_floored), 4, (0, 0, 255), 1)
        cv2.line(image, (x0_floored, y0_floored), (x1_floored,y1_floored), (0, 0, 255), 1)
    cv2.imwrite(image_path[:-4] + "_detected_test" +image_path[-4:], image)

In [7]:
def read_annotations(img_dir, edge_threshold, side_threshold):

    all_imgs = []

    # creating empty box of list of arrays where images would be later stored in
    seen_labels = {}
    # labels would be soon appended in this empty dict list

    for ann_file_name in sorted(os.listdir(img_dir)):

        # creates a list of names in the given image directory (img_dir)
        # sorted() function
        ext = os.path.splitext(ann_file_name)[1]

        # splits the pathname into the its name and its extension (basically ext is index 1 of the total file name eg:filenam.ext))

        if ext == ".txt":
            img = {"object": []}
            # instantisation

            img_file_name = img_dir + ann_file_name

            # technically contains the absolute(abs) path to the file ann_file_name

            # print("img_file_name= ", img_file_name)

            # print( "ann_file_name= ", ann_file_name)

            # read in predefined order
            file = open(img_file_name, "r")

            for line in file:

                # each line is a keypoint pair to this picture
                # file is a list of strings read from the txt file where each line is treated as a string (return or enter at the end of each line is used to determine the end of each string)
                vWords = line.split()

                # splitting the line string into a list of elements, where each element is a sub string of the main string called 'line', using whitespace as the spacer element
                obj = {}

                # an empty dict is created

                img["filename"] = os.path.splitext(img_file_name)[0] + ".png"
                # absolute path of the image file for the current annotation file being read, is stored as a string in img['filename']
                # where img is a dictionary

                if (edge_threshold < float(vWords[5])) and (
                    side_threshold < float(vWords[6])
                ):

                    obj["name"] = vWords[0]
                    # in the empty obj dict a class name is created.
                    # this name is appended by index 0 of vWords which is nothing but class

                    # the following if block is for incrementing the number of classes in the empty list of seen_lables={}
                    if obj["name"] in seen_labels:
                        seen_labels[obj["name"]] += 1
                    # now as class (0) is now visible in seen_labels for every iteration 1 is appended
                    else:
                        seen_labels[obj["name"]] = 1
                    # initally if obj['name'] is not in seen_labels then its appended by 1

                    # two keypoints which give pose (x0, y0) and direction (x1, y1) in pixel coordinates, all following coordinates are from object polygon and thus ignored
                    obj["x0"] = float(vWords[1])
                    # x0
                    obj["y0"] = float(vWords[2])
                    # y0

                    angle_alpha = (math.pi) - float(vWords[3])

                    # alpha_deg = 180-np.rad2deg(angle_alpha)
                    
                    # slope = math.tan(angle_alpha)
                    # length = 20
                    #obj["x1"] = obj["x0"]+math.sqrt(length**2/(1+slope**2))
                    #obj["y1"] = (slope*(obj["x1"]-obj["x0"])) + obj["y0"]
                    obj['x1'] = float( obj['x0'] - 20.0*math.cos(angle_alpha) )
                    obj['y1'] = float( obj['y0'] - 20.0*math.sin(angle_alpha) )

                    


                    img["object"] += [obj]

                    # the list called object is in itself stored as a key-value pair within a dictionary called img. the dict img contains two elements {'object', 'filename(abs path to img file)'}
                else:
                    print("Shaft \t",line, "was not considered for training \n")

                ##### Debug start : for visualization #####
                draw_on_image(img["filename"],img["object"])
                ##### Debug end : for visualization #####

            file.close()
            
            
            
            if len(img["object"]) > 0:
                # if the object dict is no longer empty then store all the string of data stored in img, in list of all_imgs
                all_imgs += [img]
    return all_imgs, seen_labels

In [8]:
read_annotations("F:\\dataset\\blenderdata\\",0.75,0.75)
print("\n Code ended")
winsound.Beep(freq, duration)

Shaft 	 0 123.32 147.99 -5.14 0.0 1.0 0.5
 was not considered for training 


 Code ended
